# 01 - ETL Pipeline: Social Media and Mental Health Dataset

This notebook demonstrates the ETL (Extract, Transform, Load) pipeline for the `smmh.csv` dataset.

**Objective:** Transform raw survey data into a clean, Tableau-ready dataset.

**Outputs:**
- `data/processed/v1/smmh_clean.csv`
- `docs/data_dictionary.md`
- `reports/etl_report.md`

## Setup

In [5]:
import sys
from pathlib import Path

# Add project root to path so we can import src modules
# .parent goes up one level from jupyter_notebooks/ to project root
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")

Project root: /Users/giaaxa/social-media-mental-health


In [6]:
import pandas as pd
from src.etl import run_etl, COLUMN_MAPPING, LIKERT_COLUMNS, PLATFORMS

## 1. Explore Raw Data

In [7]:
# Load raw data for exploration
raw_path = project_root / "data" / "raw" / "v1" / "smmh.csv"
df_raw = pd.read_csv(raw_path)

print(f"Shape: {df_raw.shape}")
print(f"\nColumns ({len(df_raw.columns)}):")
for i, col in enumerate(df_raw.columns, 1):
    print(f"  {i}. {col}")

Shape: (481, 21)

Columns (21):
  1. Timestamp
  2. 1. What is your age?
  3. 2. Gender
  4. 3. Relationship Status
  5. 4. Occupation Status
  6. 5. What type of organizations are you affiliated with?
  7. 6. Do you use social media?
  8. 7. What social media platforms do you commonly use?
  9. 8. What is the average time you spend on social media every day?
  10. 9. How often do you find yourself using Social media without a specific purpose?
  11. 10. How often do you get distracted by Social media when you are busy doing something?
  12. 11. Do you feel restless if you haven't used Social media in a while?
  13. 12. On a scale of 1 to 5, how easily distracted are you?
  14. 13. On a scale of 1 to 5, how much are you bothered by worries?
  15. 14. Do you find it difficult to concentrate on things?
  16. 15. On a scale of 1-5, how often do you compare yourself to other successful people through the use of social media?
  17. 16. Following the previous question, how do you feel about 

In [8]:
# Check first few rows
df_raw.head(3)

,Timestamp,1. What is your age?,2. Gender,3. Relationship Status,4. Occupation Status,5. What type of organizations are you affiliated with?,6. Do you use social media?,7. What social media platforms do you commonly use?,8. What is the average time you spend on social media every day?,9. How often do you find yourself using Social media without a specific purpose?,...,11. Do you feel restless if you haven't used Social media in a while?,"12. On a scale of 1 to 5, how easily distracted are you?","13. On a scale of 1 to 5, how much are you bothered by worries?",14. Do you find it difficult to concentrate on things?,"15. On a scale of 1-5, how often do you compare yourself to other successful people through the use of social media?","16. Following the previous question, how do you feel about these comparisons, generally speaking?",17. How often do you look to seek validation from features of social media?,18. How often do you feel depressed or down?,"19. On a scale of 1 to 5, how frequently does your interest in daily activities fluctuate?","20. On a scale of 1 to 5, how often do you face issues regarding sleep?"
0,4/18/2022 19:18:47,21.0,Male,In a relationship,University Student,University,Yes,"Facebook, Twitter, Instagram, YouTube, Discord...",Between 2 and 3 hours,5,...,2,5,2,5,2,3,2,5,4,5
1,4/18/2022 19:19:28,21.0,Female,Single,University Student,University,Yes,"Facebook, Twitter, Instagram, YouTube, Discord...",More than 5 hours,4,...,2,4,5,4,5,1,1,5,4,5
2,4/18/2022 19:25:59,21.0,Female,Single,University Student,University,Yes,"Facebook, Instagram, YouTube, Pinterest",Between 3 and 4 hours,3,...,1,2,5,4,3,3,1,4,2,5


In [9]:
# Check data types and missing values
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 21 columns):
 #   Column                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                --------------  -----  
 0   Timestamp                                                                                                             481 non-null    object 
 1   1. What is your age?                                                                                                  481 non-null    float64
 2   2. Gender                                                                                                             481 non-null    object 
 3   3. Relationship Status                                                                                                481 non-null    object 
 4   4. Occup

In [10]:
# Check unique values in key categorical columns
print("Gender values:")
print(df_raw["2. Gender"].value_counts())

print("\nTime spent values:")
print(df_raw["8. What is the average time you spend on social media every day?"].value_counts())

Gender values:
2. Gender
Female                 263
Male                   211
Nonbinary                1
Non-binary               1
NB                       1
unsure                   1
Trans                    1
Non binary               1
There are others???      1
Name: count, dtype: int64

Time spent values:
8. What is the average time you spend on social media every day?
More than 5 hours        116
Between 2 and 3 hours    101
Between 3 and 4 hours     93
Between 1 and 2 hours     70
Between 4 and 5 hours     67
Less than an Hour         34
Name: count, dtype: int64


## 2. Run ETL Pipeline

The `run_etl()` function performs all transformations and saves outputs.

In [11]:
# Change to project root for correct relative paths
import os
os.chdir(project_root)

# Run the full ETL pipeline
df_clean = run_etl()

ETL PIPELINE: Social Media and Mental Health Dataset
Loading raw data from: data/raw/v1/smmh.csv
  Loaded 481 rows, 21 columns

Running quality checks (raw)...
  Row count: 481
  Column count: 21
  Duplicate rows: 0
  Columns with missing values: 1
Renaming columns...
  Renamed 21 columns
Dropping timestamp column (privacy)...
  Dropped timestamp column
Processing age...
  Age range: 13 - 91
  Age bands: {'18-24': 309, '25-34': 77, '45+': 52, '35-44': 24, '<18': 19}
Processing gender...
  Gender clean distribution: {'Female': 263, 'Male': 211, 'Non-binary': 4, 'Unsure': 1, 'Trans': 1, 'Other': 1}
  Gender grouped distribution: {'Female': 263, 'Male': 211, 'Non-binary & Other': 7}
Converting Yes/No to boolean...
  uses_social_media: {True: 478, False: 3}
Parsing platform flags...
  Platform usage counts:
    Facebook: 407
    Twitter: 131
    Instagram: 359
    YouTube: 412
    Snapchat: 181
    Discord: 198
    Reddit: 126
    Pinterest: 145
    TikTok: 94
Parsing affiliation flags...


## 3. Inspect Cleaned Data

In [12]:
# Check new shape and columns
print(f"Shape: {df_clean.shape}")
print(f"\nColumns ({len(df_clean.columns)}):")
for col in df_clean.columns:
    print(f"  - {col}")

Shape: (481, 40)

Columns (40):
  - age
  - age_band
  - gender_clean
  - gender_grouped
  - relationship_status
  - occupation_status
  - uses_social_media
  - daily_time_band
  - daily_hours_midpoint
  - platform_facebook
  - platform_twitter
  - platform_instagram
  - platform_youtube
  - platform_snapchat
  - platform_discord
  - platform_reddit
  - platform_pinterest
  - platform_tiktok
  - platform_count
  - affil_university
  - affil_school
  - affil_company
  - affil_private
  - affil_government
  - affil_na
  - purposeless_use
  - distracted_when_busy
  - restless_without_sm
  - easily_distracted
  - worries_bother
  - difficulty_concentrating
  - compare_to_successful
  - comparison_feelings
  - seek_validation
  - low_mood_freq
  - interest_fluctuation
  - sleep_issues
  - platforms_raw
  - org_affiliations_raw
  - include_in_analysis


In [13]:
# Preview cleaned data
df_clean.head()

,age,age_band,gender_clean,gender_grouped,relationship_status,occupation_status,uses_social_media,daily_time_band,daily_hours_midpoint,platform_facebook,...,difficulty_concentrating,compare_to_successful,comparison_feelings,seek_validation,low_mood_freq,interest_fluctuation,sleep_issues,platforms_raw,org_affiliations_raw,include_in_analysis
0,21,18-24,Male,Male,In a relationship,University Student,True,Between 2 and 3 hours,2.5,1,...,5,2,3,2,5,4,5,"Facebook, Twitter, Instagram, YouTube, Discord...",University,True
1,21,18-24,Female,Female,Single,University Student,True,More than 5 hours,5.5,1,...,4,5,1,1,5,4,5,"Facebook, Twitter, Instagram, YouTube, Discord...",University,True
2,21,18-24,Female,Female,Single,University Student,True,Between 3 and 4 hours,3.5,1,...,4,3,3,1,4,2,5,"Facebook, Instagram, YouTube, Pinterest",University,True
3,21,18-24,Female,Female,Single,University Student,True,More than 5 hours,5.5,1,...,3,5,1,2,4,3,2,"Facebook, Instagram",University,True
4,21,18-24,Female,Female,Single,University Student,True,Between 2 and 3 hours,2.5,1,...,5,3,3,3,4,4,1,"Facebook, Instagram, YouTube",University,True


In [14]:
# Check data types
df_clean.dtypes

age                           Int64
age_band                     object
gender_clean                 object
gender_grouped               object
relationship_status          object
occupation_status            object
uses_social_media              bool
daily_time_band              object
daily_hours_midpoint        float64
platform_facebook             int64
platform_twitter              int64
platform_instagram            int64
platform_youtube              int64
platform_snapchat             int64
platform_discord              int64
platform_reddit               int64
platform_pinterest            int64
platform_tiktok               int64
platform_count                int64
affil_university              int64
affil_school                  int64
affil_company                 int64
affil_private                 int64
affil_government              int64
affil_na                      int64
purposeless_use               Int64
distracted_when_busy          Int64
restless_without_sm         

## 4. Validation Checks

In [15]:
# Verify Likert columns are in range 1-5
print("Likert column ranges:")
for col in LIKERT_COLUMNS:
    min_val = df_clean[col].min()
    max_val = df_clean[col].max()
    print(f"  {col}: {min_val} - {max_val}")
    assert min_val >= 1 and max_val <= 5, f"Range error in {col}"

print("\n All Likert columns validated!")

Likert column ranges:
  purposeless_use: 1 - 5
  distracted_when_busy: 1 - 5
  restless_without_sm: 1 - 5
  easily_distracted: 1 - 5
  worries_bother: 1 - 5
  difficulty_concentrating: 1 - 5
  compare_to_successful: 1 - 5
  comparison_feelings: 1 - 5
  seek_validation: 1 - 5
  low_mood_freq: 1 - 5
  interest_fluctuation: 1 - 5
  sleep_issues: 1 - 5

 All Likert columns validated!


In [16]:
# Verify no timestamp column exists
assert "timestamp" not in df_clean.columns, "Timestamp should be dropped"
print("Timestamp column successfully removed (privacy)")

Timestamp column successfully removed (privacy)


In [17]:
# Check gender grouping
print("Gender (cleaned):")
print(df_clean["gender_clean"].value_counts())
print("\nGender (grouped):")
print(df_clean["gender_grouped"].value_counts())

Gender (cleaned):
gender_clean
Female        263
Male          211
Non-binary      4
Unsure          1
Trans           1
Other           1
Name: count, dtype: int64

Gender (grouped):
gender_grouped
Female                263
Male                  211
Non-binary & Other      7
Name: count, dtype: int64


In [18]:
# Check platform flags
print("Platform usage:")
platform_cols = [f"platform_{p.lower()}" for p in PLATFORMS]
for col in platform_cols:
    count = df_clean[col].sum()
    pct = count / len(df_clean) * 100
    print(f"  {col}: {count} ({pct:.1f}%)")

Platform usage:
  platform_facebook: 407 (84.6%)
  platform_twitter: 131 (27.2%)
  platform_instagram: 359 (74.6%)
  platform_youtube: 412 (85.7%)
  platform_snapchat: 181 (37.6%)
  platform_discord: 198 (41.2%)
  platform_reddit: 126 (26.2%)
  platform_pinterest: 145 (30.1%)
  platform_tiktok: 94 (19.5%)


In [19]:
# Check analysis inclusion flag
print(f"Included in analysis: {df_clean['include_in_analysis'].sum()}")
print(f"Excluded from analysis: {(~df_clean['include_in_analysis']).sum()}")

Included in analysis: 478
Excluded from analysis: 3


## 5. Summary Statistics (for Tableau prep)

In [20]:
# Filter to analysis subset (SM users only)
df_analysis = df_clean[df_clean["include_in_analysis"]].copy()
print(f"Analysis subset: {len(df_analysis)} rows")

Analysis subset: 478 rows


In [21]:
# Descriptive stats for Likert columns
df_analysis[LIKERT_COLUMNS].describe()

,purposeless_use,distracted_when_busy,restless_without_sm,easily_distracted,worries_bother,difficulty_concentrating,compare_to_successful,comparison_feelings,seek_validation,low_mood_freq,interest_fluctuation,sleep_issues
count,478.0,478.0,478.0,478.0,478.0,478.0,478.0,478.0,478.0,478.0,478.0,478.0
mean,3.560669,3.334728,2.598326,3.359833,3.57113,3.257322,2.843096,2.776151,2.464435,3.267782,3.177824,3.213389
std,1.090977,1.319459,1.254659,1.171024,1.278004,1.342226,1.404718,1.055134,1.24629,1.307713,1.255499,1.458176
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,3.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0
50%,4.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,2.0,3.0,3.0,3.0
75%,4.0,4.0,3.0,4.0,5.0,4.0,4.0,3.0,3.0,4.0,4.0,5.0
max,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


In [22]:
# Daily time distribution
df_analysis["daily_time_band"].value_counts().sort_index()

daily_time_band
Between 1 and 2 hours     68
Between 2 and 3 hours    101
Between 3 and 4 hours     93
Between 4 and 5 hours     67
Less than an Hour         33
More than 5 hours        116
Name: count, dtype: int64

In [23]:
# Age band distribution
df_analysis["age_band"].value_counts()

age_band
18-24    308
25-34     76
45+       51
35-44     24
<18       19
Name: count, dtype: int64

## 6. Output Files

The ETL pipeline has generated:

1. **`data/processed/v1/smmh_clean.csv`** - Clean dataset for Tableau
2. **`docs/data_dictionary.md`** - Column reference documentation
3. **`reports/etl_report.md`** - ETL summary with quality checks

In [24]:
# Verify output files exist
outputs = [
    project_root / "data" / "processed" / "v1" / "smmh_clean.csv",
    project_root / "docs" / "data_dictionary.md",
    project_root / "reports" / "etl_report.md",
]

for path in outputs:
    exists = path.exists()
    status = "" if exists else ""
    print(f"{status} {path.relative_to(project_root)}")

 data/processed/v1/smmh_clean.csv
 docs/data_dictionary.md
 reports/etl_report.md


---

**Next steps:**
- Load `smmh_clean.csv` into Tableau
- Set default filter: `include_in_analysis = True`
- Begin exploratory analysis